In [1]:
# Concept for DASK + POLARS for online large sequencing dataset handling

import polars as pl
import pandas as pd
import numpy as np
import pathlib
import glob
from dask.distributed import Client, LocalCluster, wait
import dask.dataframe as dd
from pyarrow.dataset import dataset


# sorted(glob.glob("./Input/KJS/EYS_2nd_lib/**/*Frags.fastq", recursive=True))
INPUT_DATA = ['../data/former_structure/Input/KJS/EYS_2nd_lib/1/20230717_1.extendedFrags.fastq',
#  '../data/former_structure/Input/KJS/EYS_2nd_lib/2/20230717_2.extendedFrags.fastq',
#  '../data/former_structure/Input/KJS/EYS_2nd_lib/3/20230717_3.extendedFrags.fastq',
#  '../data/former_structure/Input/KJS/EYS_2nd_lib/4/20230717_4.extendedFrags.fastq',
#  '../data/former_structure/Input/KJS/EYS_2nd_lib/5/20230717_5.extendedFrags.fastq',
#  '../data/former_structure/Input/KJS/EYS_2nd_lib/6/20230717_6.extendedFrags.fastq',
#  '../data/former_structure/Input/KJS/EYS_2nd_lib/7/20230717_7.extendedFrags_error_truncated.fastq',
#  '../data/former_structure/Input/KJS/EYS_2nd_lib/8/20230717_8.extendedFrags.fastq'
 ]

In [2]:
cluster = LocalCluster()
client = cluster.get_client()
client

/mnt/P41/Repositories/extractor_SKKUGE/.conda/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38109 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:38109/status,
Dashboard: http://127.0.0.1:38109/status,Workers: 8
Total threads: 32,Total memory: 62.67 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43701,Workers: 8
Dashboard: http://127.0.0.1:38109/status,Total threads: 32
Started: Just now,Total memory: 62.67 GiB
Comm: tcp://127.0.0.1:33113,Total threads: 4
Dashboard: http://127.0.0.1:41315/status,Memory: 7.83 GiB
Nanny: tcp://127.0.0.1:40659,


In [3]:
ddf = dd.read_csv(INPUT_DATA[0], header=None, names=["DATA"], blocksize="0.25GB")
print(ddf.npartitions)


106


In [4]:
ddf_length = len(ddf)
print(ddf_length)
if ddf_length % 4 != 0:
    raise ValueError("The number of rows in the array is not a multiple of 4!")
ddf_reshaped = ddf["DATA"].to_dask_array(lengths=True).reshape(-1, 4)
ddf_reshaped

179206932


/mnt/P41/Repositories/extractor_SKKUGE/.conda/lib/python3.11/site-packages/dask/dataframe/core.py:3830: UserWarning: Dask currently has limited support for converting pandas extension dtypes to arrays. Converting string to object dtype.
  warnings.warn(


dask.array<reshape, shape=(44801733, 4), dtype=object, chunksize=(423410, 4), chunktype=numpy.ndarray>

In [5]:
ddf_reshaped = ddf_reshaped.to_dask_dataframe(columns=["ID", "SEQUENCE", "+", "QUALITY"])
ddf_reshaped

,ID,SEQUENCE,+,QUALITY
npartitions=106,,,,
0,string,string,string,string
420485,...,...,...,...
...,...,...,...,...
44378965,...,...,...,...
44801732,...,...,...,...


In [6]:
ddf_reshaped = ddf_reshaped.drop(columns=["+"])
ddf_reshaped.head()

,ID,SEQUENCE,QUALITY
0,@ST-E00127:1400:HJTMMCCX2:2:1101:18954:3102 1:...,GTTATCAACTTGAAAAAGTGGCACCGAGTCGGCGCACAGTTTAAAT...,AAAFFJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJFJJJJJ...
1,@ST-E00127:1400:HJTMMCCX2:2:1101:19705:3102 1:...,GTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGCATGGAACTATT...,AAFFFJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJFJJJJFJ<FJJ...
2,@ST-E00127:1400:HJTMMCCX2:2:1101:20983:3102 1:...,GTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGCGCAAAATTGTC...,AAFFFJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ<JJFF<JFJ7...
3,@ST-E00127:1400:HJTMMCCX2:2:1101:22201:3102 1:...,GTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGCAATTACTTGAT...,AAFFFJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJA-FJF7FJJ-<...
4,@ST-E00127:1400:HJTMMCCX2:2:1101:23175:3102 1:...,GTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGCACAAACAGATT...,AAFFFJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJFFJA--7--<7...


In [7]:
parquet_dir_path = pathlib.Path(INPUT_DATA[0]).parent / "parquet"

In [8]:
ddf_reshaped.to_parquet(
    parquet_dir_path, engine="pyarrow", compression="snappy", write_index=False
)

In [9]:
pyarrow_dataset = dataset(
    source=parquet_dir_path ,
    format="parquet",
)
pyarrow_dataset

In [10]:
ldf = pl.scan_pyarrow_dataset(pyarrow_dataset)
ldf.select(pl.count()).collect(streaming=True)

count
u32
44801733


In [11]:
# qs = [pl.scan_csv(f, has_header=False) for i, f in enumerate(INPUT_DATA)]

In [12]:
# q= qs[0]


In [13]:
sample_query = (
    ldf.select(["SEQUENCE"])
    .with_columns(
        pl.col("SEQUENCE").str.contains(
            r"TTTTACTACACATCGCTACTACTG(.*)TCATCCTTGGAAGAATCCATTACCCTATCAAAGTAATTTG"
        )
    )
    .sum()
    .collect(streaming=True)
)
sample_query

SEQUENCE
u32
540


In [14]:
sample_query = (
    ldf.select(["SEQUENCE"])
    .with_columns(
        pl.col("SEQUENCE").str.contains(
            r"TTTTACTACACATCGCTACTACTG"
        )
    )
    .sum()
    .collect(streaming=True)
)
sample_query

SEQUENCE
u32
636


In [15]:
# [
#     q.with_columns(
#         pl.col(q.columns[0]).str.contains(
#             r"TTTTACTACACATCGCTACTACTG(.*)TCATCCTTGGAAGAATCCATTACCCTATCAAAGTAATTTG"
#         )
#     )
#     .sum()
#     .collect(streaming=True)
#     .to_numpy()
#     .squeeze()
#     .item()
#     for q in qs
# ]

In [16]:
# [
#     q.with_columns(
#         pl.col(q.columns[0]).str.contains(
#             r"TTTTACTACACATCGCTACTACTG"
#         )
#     )
#     .sum()
#     .collect(streaming=True)
#     .to_numpy()
#     .squeeze()
#     .item()
#     for q in qs
# ]